In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import os
import time

# --- 0. Chuẩn bị Thư mục và Thiết bị ---
MODEL_DIR = 'models'
os.makedirs(MODEL_DIR, exist_ok=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Thư mục lưu mô hình đã sẵn sàng: {MODEL_DIR}")
print(f"Sử dụng thiết bị: {device}")

# --- Dữ liệu Giả định (Không Chuẩn hóa) ---
N_SAMPLES = 1000
N_FEATURES = 1 
TIMESTEPS = 20 

# Tạo dữ liệu giá tiền ảo giả định có biến động lớn
np.random.seed(42)
base_price = np.cumsum(np.random.randn(N_SAMPLES + TIMESTEPS) * 0.1) + 100
# Thêm các 'dị thường' (biến động lớn) ngẫu nhiên
anomaly_indices = np.random.randint(N_SAMPLES // 2, N_SAMPLES, size=10)
base_price[anomaly_indices] += np.random.uniform(5, 10, size=10) # Dị thường lớn
raw_data = base_price.reshape(-1, 1).astype(np.float32)

# --- 1. Hàm Split Data và Tạo Sequence cho LSTM ---
def create_sequences(data, timesteps):
    """Tạo X (sequence) và y (target) cho mô hình LSTM."""
    Xs, ys = [], []
    for i in range(len(data) - timesteps):
        # Lấy một cửa sổ (window) dữ liệu
        X = data[i:(i + timesteps), :]
        # Lấy giá trị tiếp theo làm target cho dự đoán (Model 1)
        y_forecast = data[i + timesteps, :]
        
        Xs.append(X)
        ys.append(y_forecast)
        
    return np.array(Xs), np.array(ys)

# Chia Train/Validation/Test (70/15/15)
train_size = int(N_SAMPLES * 0.7)
val_size = int(N_SAMPLES * 0.15)

# Chia tập dữ liệu
train_data = raw_data[:train_size + TIMESTEPS]
val_data = raw_data[train_size:train_size + val_size + TIMESTEPS]

# Tạo sequences 
X_train_np, y_train_forecast_np = create_sequences(train_data, TIMESTEPS)
X_val_np, y_val_forecast_np = create_sequences(val_data, TIMESTEPS)

print("\n--- Kích thước Dữ liệu sau khi chia (Không Chuẩn hóa) ---")
print(f"X_train shape: {X_train_np.shape}")
print(f"y_train_forecast shape: {y_train_forecast_np.shape}")


# --- 2. PyTorch Dataset và DataLoader ---

class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).float()
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

BATCH_SIZE = 32

# Model 1 (Dự đoán) DataLoaders
train_dataset_forecast = TimeSeriesDataset(X_train_np, y_train_forecast_np)
train_loader_forecast = DataLoader(train_dataset_forecast, batch_size=BATCH_SIZE, shuffle=False)

val_dataset_forecast = TimeSeriesDataset(X_val_np, y_val_forecast_np)
val_loader_forecast = DataLoader(val_dataset_forecast, batch_size=BATCH_SIZE, shuffle=False)

# Model 2 (Autoencoder) DataLoaders (Input và Target là X)
train_dataset_anomaly = TimeSeriesDataset(X_train_np, X_train_np)
train_loader_anomaly = DataLoader(train_dataset_anomaly, batch_size=BATCH_SIZE, shuffle=False)

val_dataset_anomaly = TimeSeriesDataset(X_val_np, X_val_np)
val_loader_anomaly = DataLoader(val_dataset_anomaly, batch_size=BATCH_SIZE, shuffle=False)


# --- 3. Định nghĩa các Mô hình PyTorch LSTM ---

## Mô hình 1: Dự đoán Chuỗi thời gian
class LSTMForecast(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMForecast, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        # x shape: (batch_size, seq_len, input_size)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Output, (h_n, c_n)
        out, _ = self.lstm(x, (h0, c0))
        # Lấy output của bước thời gian cuối cùng
        out = self.fc(out[:, -1, :]) 
        return out

## Mô hình 2: Phát hiện Bất thường (LSTM Autoencoder)
class LSTMAutoencoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTMAutoencoder, self).__init__()
        self.timesteps = TIMESTEPS
        
        # Encoder
        self.encoder = nn.LSTM(input_size, hidden_size, 2, batch_first=True, dropout=0.2)
        
        # Decoder
        self.decoder = nn.LSTM(hidden_size, input_size, 2, batch_first=True, dropout=0.2)
        
    def forward(self, x):
        # x shape: (batch_size, timesteps, input_size)
        
        # 1. Encoder: Học cách nén
        # out_e: output cho từng bước thời gian, (h_n, c_n): hidden/cell state cuối cùng
        _, (hidden_state, cell_state) = self.encoder(x)
        
        # 2. Decoder Input: Lấy hidden state cuối cùng của Encoder, nhân bản cho mọi bước thời gian
        # Kích thước hidden_state: (num_layers, batch_size, hidden_size)
        # Chúng ta cần hidden state của layer cuối cùng: hidden_state[-1, :, :]
        decoder_input = hidden_state[-1, :, :].unsqueeze(1).repeat(1, self.timesteps, 1)
        # decoder_input shape: (batch_size, timesteps, hidden_size)
        
        # 3. Decoder: Học cách tái tạo
        # Ở đây chúng ta muốn tái tạo lại chuỗi input ban đầu. 
        # Cần điều chỉnh kiến trúc Decoder hoặc sử dụng TimeDistributed.
        # Để đơn giản và phù hợp với cách Keras Autoencoder, ta dùng hidden state làm input cho Decoder
        
        # Thay vì decoder_input phức tạp, dùng cách đơn giản hơn:
        # Giữ lại hidden state và cell state cuối cùng của Encoder
        # Sử dụng chúng để khởi tạo Decoder
        
        # Tạo Tensor zero có kích thước (batch_size, timesteps, input_size)
        # Vì decoder sẽ output ra chính input_size
        decoder_input_tensor = torch.zeros(x.size(0), self.timesteps, self.encoder.hidden_size).to(x.device)

        # Giữ lại (h_n, c_n) của encoder
        # Sử dụng output của encoder làm đầu vào cho Decoder. 
        # Vì Decoder cần tái tạo lại chuỗi T bước, ta cần một bước trung gian.
        
        # Cách tiếp cận Autoencoder đơn giản:
        # Output của Encoder (hidden_state[-1]) -> RepeatVector -> Decoder (tái tạo)
        
        # Input cho Decoder: hidden_state cuối cùng (layer cuối, mọi batch)
        # shape: (batch_size, hidden_size)
        latent_vector = hidden_state[-1]
        
        # Lặp lại vector nén T lần
        decoder_input = latent_vector.unsqueeze(1).repeat(1, self.timesteps, 1)
        
        # Decoder: cần LSTM với hidden_size = input_size và output_size = input_size
        # Ta sẽ dùng nn.Linear sau mỗi bước của decoder.
        
        # Khởi tạo lại Decoder đơn giản hơn:
        
        out_dec, _ = self.decoder(decoder_input)
        
        # Áp dụng Linear layer tại mỗi bước thời gian
        # Vì Output của LSTM là (batch_size, timesteps, input_size)
        # Ta cần nn.Linear(input_size, N_FEATURES)
        
        final_output = nn.Linear(input_size, N_FEATURES).to(x.device)(out_dec)
        
        return final_output


# --- 4. Hàm Training (Tích hợp Early Stopping) ---

def train_model(model, train_loader, val_loader, loss_fn, optimizer, epochs, model_name):
    print(f"\n#################################################################")
    print(f"### BẮT ĐẦU HUẤN LUYỆN MÔ HÌNH: {model_name} ###")
    print(f"#################################################################")
    
    # Early Stopping Parameters
    patience = 10
    min_delta = 1e-4
    best_val_loss = float('inf')
    epochs_no_improve = 0
    history = {'loss': [], 'val_loss': []}
    
    for epoch in range(epochs):
        start_time = time.time()
        
        # --- Training Loop ---
        model.train()
        total_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            optimizer.zero_grad()
            output = model(X_batch)
            
            loss = loss_fn(output, y_batch)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        avg_train_loss = total_loss / len(train_loader)
        
        # --- Validation Loop ---
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for X_val, y_val in val_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)
                
                output = model(X_val)
                val_loss = loss_fn(output, y_val)
                total_val_loss += val_loss.item()
                
        avg_val_loss = total_val_loss / len(val_loader)
        
        history['loss'].append(avg_train_loss)
        history['val_loss'].append(avg_val_loss)

        # In ra loss (theo yêu cầu)
        print(f"Epoch {epoch+1:02d} ({time.time() - start_time:.2f}s): Loss={avg_train_loss:.4f}, Val_Loss={avg_val_loss:.4f}")

        # --- Early Stopping Check ---
        if avg_val_loss < best_val_loss - min_delta:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
            # Lưu checkpoint mô hình tốt nhất
            torch.save(model.state_dict(), os.path.join(MODEL_DIR, f'{model_name}_best_weights.pth'))
        else:
            epochs_no_improve += 1
            
        if epochs_no_improve == patience:
            print(f"⚠️ Early stopping tại Epoch {epoch+1:02d}!")
            # Tải lại trọng số tốt nhất trước khi dừng
            model.load_state_dict(torch.load(os.path.join(MODEL_DIR, f'{model_name}_best_weights.pth')))
            break
            
    return model, history


# --- 5. Huấn luyện Mô hình 1 (Dự đoán) ---
model_forecast = LSTMForecast(
    input_size=N_FEATURES, 
    hidden_size=64, 
    num_layers=2, 
    output_size=N_FEATURES
).to(device)

loss_fn_forecast = nn.MSELoss() # Dùng MSE cho dự đoán giá trị liên tục
optimizer_forecast = torch.optim.Adam(model_forecast.parameters(), lr=1e-3)

model_forecast, history_forecast = train_model(
    model_forecast, 
    train_loader_forecast, 
    val_loader_forecast, 
    loss_fn_forecast, 
    optimizer_forecast, 
    epochs=100, 
    model_name='lstm_forecast'
)

# Lưu mô hình cuối cùng (đã khôi phục trọng số tốt nhất)
torch.save(model_forecast.state_dict(), os.path.join(MODEL_DIR, 'lstm_forecast_model.pth'))
print(f"\n✅ Mô hình 1 đã được lưu tại: {os.path.join(MODEL_DIR, 'lstm_forecast_model.pth')}")


# --- 6. Huấn luyện Mô hình 2 (Bất thường - Autoencoder) ---
# PyTorch Autoencoder phức tạp hơn Keras, ta sẽ đơn giản hóa đầu ra của Decoder để phù hợp.
model_anomaly = LSTMAutoencoder(
    input_size=N_FEATURES, 
    hidden_size=64 # hidden_size phải đủ lớn để chứa thông tin nén
).to(device)

loss_fn_anomaly = nn.L1Loss() # Dùng L1Loss (MAE) để đánh giá lỗi tái tạo
optimizer_anomaly = torch.optim.Adam(model_anomaly.parameters(), lr=1e-3)

model_anomaly, history_anomaly = train_model(
    model_anomaly, 
    train_loader_anomaly, 
    val_loader_anomaly, 
    loss_fn_anomaly, 
    optimizer_anomaly, 
    epochs=100, 
    model_name='lstm_anomaly_autoencoder'
)

# Lưu mô hình cuối cùng (đã khôi phục trọng số tốt nhất)
torch.save(model_anomaly.state_dict(), os.path.join(MODEL_DIR, 'lstm_anomaly_autoencoder.pth'))
print(f"\n✅ Mô hình 2 đã được lưu tại: {os.path.join(MODEL_DIR, 'lstm_anomaly_autoencoder.pth')}")

print("\n🎉 Hoàn tất việc xây dựng, huấn luyện và lưu cả hai mô hình bằng PyTorch!")


# --- Phần bổ sung: Tải Mô hình ---
print("\n--- Phần bổ sung: Tải mô hình đã lưu ---")

# Tải Mô hình 1 (Dự đoán)
loaded_model_forecast = LSTMForecast(N_FEATURES, 64, 2, N_FEATURES).to(device)
loaded_model_forecast.load_state_dict(torch.load(os.path.join(MODEL_DIR, 'lstm_forecast_model.pth')))
loaded_model_forecast.eval()
print("✅ Tải Mô hình 1 thành công.")

# Tải Mô hình 2 (Bất thường)
loaded_model_anomaly = LSTMAutoencoder(N_FEATURES, 64).to(device)
loaded_model_anomaly.load_state_dict(torch.load(os.path.join(MODEL_DIR, 'lstm_anomaly_autoencoder.pth')))
loaded_model_anomaly.eval()
print("✅ Tải Mô hình 2 thành công.")   